### Purpose: Handle Missing Data
Handle the missing data in building metadata, weather train files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import warnings
import datetime

In [ ]:
# first create Filled Weather Data

weather_dataframe = pd.read_csv('drive/MyDrive/9417_team_folder/team_folder/data/weather_train.csv')
print(len(weather_dataframe))


139773


In [ ]:
timestamp = "%Y-%m-%d %H:%M:%S"
begin_time = datetime.datetime.strptime(weather_dataframe['timestamp'].min(),timestamp)
finish_time = datetime.datetime.strptime(weather_dataframe['timestamp'].max(),timestamp)

total_hours = int(((finish_time - begin_time).total_seconds() + 3600) / 3600)
all_timestamps = [(finish_time - datetime.timedelta(hours=x)).strftime(timestamp) for x in range(total_hours)]

for sid in range(16):
    #store array with all timestamps to that site_id
    site_time = np.array(weather_dataframe[weather_dataframe['site_id'] == sid]['timestamp'])

    #Concatenate all timestamps NOT in the iterated site time
    differ_time = pd.DataFrame(np.setdiff1d(all_timestamps, site_time), columns=['timestamp'])

    #create new datafram including all this excluded timestamps, and cocatenate
    differ_time['site_id'] = sid
    weather_dataframe = pd.concat([weather_dataframe,differ_time])

weather_dataframe = weather_dataframe.reset_index(drop=True)    

In [ ]:
# Find number of missing data 
"""
def missing_data(df):
  missing_data_df = pd.DataFrame(df.isnull().sum()).reset_index()
  missing_data_df.columns=['Feature Name',"no. missing values"]
  missing_data_df['no. rows'] = df.shape[0]
  missing_data_df['% missed'] = round((missing_data_df['no. missing values']/missing_data_df['no. rows'])*100,2)
  return missing_data_df"""

'\ndef missing_data(df):\n  missing_data_df = pd.DataFrame(df.isnull().sum()).reset_index()\n  missing_data_df.columns=[\'Feature Name\',"no. missing values"]\n  missing_data_df[\'no. rows\'] = df.shape[0]\n  missing_data_df[\'% missed\'] = round((missing_data_df[\'no. missing values\']/missing_data_df[\'no. rows\'])*100,2)\n  return missing_data_df'

In [ ]:
def missing_data(df):
  missing_data_df = df.isnull().sum().reset_index()
  missing_data_df.columns = ['Feature Name', 'No. Missing Values']
  n_rows = df.shape[0]
  missing_data_df['% Missing'] = round(missing_data_df['No. Missing Values']/(n_rows*100, 2))
  missing_data_df['No. Rows'] = n_rows
  missing_data_df = missing_data_df[['Feature Name', 'No. Missing Values', 'No. Rows', '% Missing']]

  return missing_data_df


#### Calculate number of missing values, missing percentage for each feature in weather

In [11]:
missing_data(weather_dataframe)

ValueError: ignored

### Fill in new data based on day, week, month features.


In [ ]:
## rearrange code
weather_dataframe["datetime"] = pd.to_datetime(weather_dataframe["timestamp"])
weather_dataframe["s_day"] = weather_dataframe["datetime"].dt.day
weather_dataframe["s_week"] = weather_dataframe["datetime"].dt.week
weather_dataframe["s_month"] = weather_dataframe["datetime"].dt.month
weather_dataframe = weather_dataframe.set_index(['site_id','s_day','s_month'])

<ipython-input-7-e4777db36946>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  weather_dataframe["s_week"] = weather_dataframe["datetime"].dt.week


### Fill the missing values using the average temperature in that given month.

In [ ]:
def complete_missing_weather(weather_dataframe):

  # Air Temperature
  air_temp = pd.DataFrame(weather_dataframe.groupby(['site_id','s_day','s_month'])['air_temperature'].mean(),columns=["air_temperature"])
  weather_dataframe.update(air_temp,overwrite=False)

  # Cloud Coverage
  cloud_cov = weather_dataframe.groupby(['site_id','s_day','s_month'])['cloud_coverage'].mean()
  cloud_cov = pd.DataFrame(cloud_cov.fillna(method='ffill'),columns=["cloud_coverage"])
  weather_dataframe.update(cloud_cov ,overwrite=False)

  # Dew Temperature
  dew_temp = pd.DataFrame(weather_dataframe.groupby(['site_id','s_day','s_month'])['dew_temperature'].mean(),columns=["dew_temperature"])
  weather_dataframe.update(dew_temp,overwrite=False)

  # Sea-Level
  sea_level = weather_dataframe.groupby(['site_id','s_day','s_month'])['sea_level_pressure'].mean()
  sea_level = pd.DataFrame(sea_level.fillna(method='ffill'),columns=['sea_level_pressure'])
  weather_dataframe.update(sea_level,overwrite=False)

  # Wind Direction
  wind_dir = pd.DataFrame(weather_dataframe.groupby(['site_id','s_day','s_month'])['wind_direction'].mean(), columns=['wind_direction'])
  weather_dataframe.update(wind_dir, overwrite=False)

  # Wind Speed
  wind_speed =  pd.DataFrame(weather_dataframe.groupby(['site_id','s_day','s_month'])['wind_speed'].mean(),columns=['wind_speed'])
  weather_dataframe.update(wind_speed,overwrite=False)

  # Precipitation Depth
  precip_depth = weather_dataframe.groupby(['site_id','s_day','s_month'])['precip_depth_1_hr'].mean()
  precip_depth = pd.DataFrame(precip_depth.fillna(method='ffill'),columns=['precip_depth_1_hr'])
  weather_dataframe.update(precip_depth,overwrite=False)

  # Reset weather variables, drop joined columns
  weather_dataframe = weather_dataframe.reset_index()
  weather_dataframe = weather_dataframe.drop(['datetime','s_day','s_week','s_month'],axis=1)
      
  return weather_dataframe


In [12]:
weather_train_complete = complete_missing_weather(weather_dataframe)
# missing_data(weather_train_complete)

In [14]:
weather_train_complete.to_csv('weather_train_filled.csv')
print(weather_train_complete.head())

   site_id            timestamp  air_temperature  cloud_coverage  \
0        0  2016-01-01 00:00:00             25.0        6.000000   
1        0  2016-01-01 01:00:00             24.4        4.285714   
2        0  2016-01-01 02:00:00             22.8        2.000000   
3        0  2016-01-01 03:00:00             21.1        2.000000   
4        0  2016-01-01 04:00:00             20.0        2.000000   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0             20.0          -0.173913              1019.7             0.0   
1             21.1          -1.000000              1020.2            70.0   
2             21.1           0.000000              1020.2             0.0   
3             20.6           0.000000              1020.1             0.0   
4             20.0          -1.000000              1020.0           250.0   

   wind_speed  
0         0.0  
1         1.5  
2         0.0  
3         0.0  
4         2.6  


## Combine all trained data

In [ ]:
data_dir = "drive/MyDrive/team_folder/data/"

train_raw = pd.read_csv(data_dir + "train.csv", sep = ',')
weather_train_raw = pd.read_csv(data_dir + "weather_train_filled.csv", sep = ',')
building_meta_raw = pd.read_csv(data_dir + "building_metadata.csv", sep = ',')

### Step 1: Group the primary_use category and 1-hot encode

In [ ]:
other_categories = ['Other', 'Healthcare', 'Parking', 'Warehouse/storage', 
                    'Manufacturing/industrial', 'Retail', 'Services', 
                    'Technology/science', 'Food sales and service', 
                    'Utility', 'Religious worship']

building_meta_raw.loc[building_meta_raw['primary_use'].isin(other_categories), 'primary_use'] = 'Other'
print(building_meta_raw.primary_use.value_counts())

In [ ]:
# This line encodes to one hot
building_meta_raw = pd.get_dummies(building_meta_raw)

In [ ]:
merged_train_raw = train_raw.merge(building_meta_raw, on='building_id', how='left')
merged_train_raw = merged_train_raw.merge(weather_train_raw, on=['site_id', 'timestamp'], how='left')

Step 2: Remove floor count and year built

In [ ]:
merged_train_raw = merged_train_raw.drop(columns=['floor_count', 'year_built', 'Unnamed: 0'])
print(merged_train_raw)

Step 3: Convert site0 meter_reading from kBTU to kWh

In [ ]:
kBTU_conversion_coef = 0.2931
merged_train_raw.loc[merged_train_raw['site_id'] == 0, 'meter_reading'] *= kBTU_conversion_coef

In [ ]:
merged_train_raw.to_csv(data_dir + 'custom_data/categorical_train_set.csv', index=False)

In [ ]:
#!mv weather_train_filled.csv drive/MyDrive/Colab\ Notebooks/COMP9417/group_project/team_folder/data/

In [ ]:
!ls